In [21]:
import pandas as pd
import matplotlib
import seaborn as sn
import plotly.express as px
import world_bank_data as wb

## Data Imports

In [140]:
df_sovereign_projects = pd.read_excel("__data/adb_sovereign_projects_all.xlsx")
df_sovereign_projects.head(1)

,Project Number,Project Name,Country,Financing,Project Type or Modality,Status,Approval Number,Approval Date,ADB Financing (US$),Geographical Location,Sector,Subsector,Safeguard - Environment,Safeguard - Indigenous People,Safeguard - Resettlement,Department,Executing Agency,Project Officer,Description
0,31624-023,Melamchi Water Supply Project,Nepal,Sovereign,Loan,Active,1820 | 3110 | 8191 | 8235,2008-07-03,145000000.0,"Ambathan, Bagmati Zone, Bansbari, Bhotechaur, ...",Water and other urban infrastructure and services,"Urban policy, institutional and capacity devel...",A,C,A,South Asia Department,Kathmandu Upatyaka Khanepani Limited (KUKL),"Dasgupta, Saugata",The main objectives of the Project are to alle...


In [141]:
# get year column
df_sovereign_projects['year_approved'] = df_sovereign_projects['Approval Date'].dt.year

# Filter year > 2015
df_sovereign_projects = df_sovereign_projects[df_sovereign_projects['year_approved'] > 2015]

# Replace some country names
df_sovereign_projects[df_sovereign_projects['Country'] == "China, People's Republic of"] = "China"
df_sovereign_projects[df_sovereign_projects['Country'] == "Viet Nam"] = "Vietnam"
df_sovereign_projects[df_sovereign_projects['Country'] == "Lao People's Democratic Republic"] = "Lao PDR"
df_sovereign_projects[df_sovereign_projects['Country'] == "Micronesia, Federated States of"] = "Micronesia, Fed. Sts."

In [142]:
# merge world bank region names
df_wb_countries = wb.get_countries()[['name', 'region']].reset_index().sort_values(by="name")
df_sovereign_projects = pd.merge(df_sovereign_projects, df_wb_countries, left_on="Country", right_on="name", how="left")

## Funding by Region

In [137]:
df_sovereign_projects_region = df_sovereign_projects[['year_approved', 'region', 'ADB Financing (US$)']].groupby(['year_approved', 'region']).agg('sum').reset_index()
df_sovereign_projects_region

,year_approved,region,ADB Financing (US$)
0,2016.0,East Asia & Pacific,2529051000.0
1,2016.0,Europe & Central Asia,3455980000.0
2,2016.0,South Asia,9798118345.0
3,2017.0,East Asia & Pacific,4987175000.0
4,2017.0,Europe & Central Asia,2167780000.0
5,2017.0,South Asia,11746105000.0
6,2018.0,East Asia & Pacific,4445055000.0
7,2018.0,Europe & Central Asia,1797175000.0
8,2018.0,South Asia,6773139000.0
9,2019.0,East Asia & Pacific,8010425000.0


In [138]:
df_sovereign_projects_region = df_sovereign_projects_region[['year_approved', "ADB Financing (US$)", "region"]]
df_sovereign_projects_region = df_sovereign_projects_region[df_sovereign_projects_country['year_approved'].isin([2021, 2020, 2019, 2018, 2017, 2016])]


In [139]:
fig = px.bar(df_sovereign_projects_region, x="year_approved", y="ADB Financing (US$)", color="region", title="Project Funding by Region")
fig.show()

## Funding by Sector

In [212]:
df_sovereign_projects_sector = df_sovereign_projects[['year_approved', 'Sector', 'ADB Financing (US$)']]

df_sovereign_projects_sector['Sector'] = df_sovereign_projects_sector['Sector'].str.split("|", expand=True).iloc[:,0].str.strip()

df_sovereign_projects_sector = df_sovereign_projects_sector.groupby(['year_approved', 'Sector']).agg('sum').reset_index()

df_sovereign_projects_sector = df_sovereign_projects_sector[df_sovereign_projects_sector['year_approved'].isin([2021, 2020, 2019, 2018, 2017, 2016])]

df_sovereign_projects_sector.head(1)

C:\Users\Bened\AppData\Local\Temp\ipykernel_24632\1903799695.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year_approved,Sector,ADB Financing (US$)
0,2016.0,Agriculture and Natural Resources,0.0


In [215]:
fig = px.bar(df_sovereign_projects_sector, x="year_approved", y="ADB Financing (US$)", color="Sector", title="Project Funding by Sector")
fig.show()

## Funding by Funding Type

In [175]:
df_sovereign_projects_loantype = df_sovereign_projects[['year_approved', 'Project Type or Modality', 'ADB Financing (US$)']]

df_sovereign_projects_loantype['Project Type or Modality'] = df_sovereign_projects_loantype['Project Type or Modality'].str.split("|", expand=True).iloc[:,0].str.strip()

df_sovereign_projects_loantype = df_sovereign_projects_loantype.groupby(['year_approved', 'Project Type or Modality']).agg('sum').reset_index()

df_sovereign_projects_loantype = df_sovereign_projects_loantype[df_sovereign_projects_loantype['year_approved'].isin([2021, 2020, 2019, 2018, 2017, 2016])]

df_sovereign_projects_loantype.head(1)

C:\Users\Bened\AppData\Local\Temp\ipykernel_24632\3269479675.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,year_approved,Project Type or Modality,ADB Financing (US$)
0,2016.0,Grant,4172597345.0


In [178]:
fig = px.bar(df_sovereign_projects_loantype, x="year_approved", y="ADB Financing (US$)", color="Project Type or Modality", title="Project Funding by Funding Type")
fig.show()

## Funding Winners/Loosers COVID

In [237]:
df_sovereign_projects_countrydiff = df_sovereign_projects[['year_approved', 'name', 'ADB Financing (US$)']]

df_sovereign_projects_countrydiff = df_sovereign_projects_countrydiff[df_sovereign_projects_countrydiff['year_approved'].isin([2020, 2019])]

df_sovereign_projects_countrydiff = df_sovereign_projects_countrydiff.groupby(['year_approved', 'name']).agg("sum").reset_index()

df_sovereign_projects_countrydiff = df_sovereign_projects_countrydiff.pivot(index="name", columns="year_approved", values="ADB Financing (US$)").reset_index()

df_sovereign_projects_countrydiff['diff'] = df_sovereign_projects_countrydiff[2020] / df_sovereign_projects_countrydiff[2019]

df_sovereign_projects_countrydiff = df_sovereign_projects_countrydiff.dropna()

df_sovereign_projects_countrydiff = pd.concat([df_sovereign_projects_countrydiff.nlargest(n = 10,columns=['diff']), df_sovereign_projects_countrydiff.nsmallest(n=10,columns=['diff'])], keys=['Winner', 'Looser']).sort_values('diff').reset_index()

df_sovereign_projects_countrydiff['eval'] = df_sovereign_projects_countrydiff['level_0']

df_sovereign_projects_countrydiff = df_sovereign_projects_countrydiff.round(2)

df_sovereign_projects_countrydiff


year_approved,level_0,level_1,name,2019.0,2020.0,diff,eval
0,Looser,2,Azerbaijan,2.509000e+08,1.580000e+06,0.01,Looser
1,Looser,1,Armenia,5.197000e+07,4.180000e+06,0.08,Looser
2,Looser,17,Nauru,2.200000e+07,5.000000e+06,0.23,Looser
3,Looser,24,Solomon Islands,6.775000e+07,2.000000e+07,0.30,Looser
4,Looser,12,Kyrgyz Republic,1.899750e+08,7.367500e+07,0.39,Looser
5,Looser,19,Pakistan,2.136582e+09,1.423100e+09,0.67,Looser
6,Looser,15,Mongolia,5.245150e+08,3.786820e+08,0.72,Looser
7,Looser,7,Georgia,7.127750e+08,5.240750e+08,0.74,Looser
8,Looser,0,Afghanistan,3.663556e+08,2.901750e+08,0.79,Looser
9,Looser,22,Philippines,5.073300e+09,4.082533e+09,0.80,Looser


In [238]:
px.bar(df_sovereign_projects_countrydiff, y = "name", x = "diff", color='eval', color_discrete_sequence=['red', 'green'], title="Top-10 Winners/Loosers COVID Shock<br><sup>Numbers are referring to the 2020 funding divided by the 2019 funding received per country</sup>", text_auto=True, height=700)